In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import syft as sy
import numpy as np
import pandas as pd
from syft.core.adp.entity_list import EntityList
from time import time

In [3]:
t0 = time()
# df = pd.read_parquet("/home/ruchi/1B_rows_dataset_sample.parquet")
df = pd.read_parquet("../Ishan/1M_rows_dataset_sample.parquet")
tf = time() - t0

print(f"Time taken to read file with {df.shape[0]/1e6} million rows : {tf} seconds")

Time taken to read file with 1.0 million rows : 0.20961976051330566 seconds


In [4]:
scale = 100  # This is put here to stop int32 overflow errors


t0 = time()
impressions = df['impressions'].to_numpy(np.int32) // scale
data_subjects = EntityList.from_series(df['user_id'])
tf = time() - t0

print(f"Time taken to create inputs for Syft Tensor: {tf} seconds")

Time taken to create inputs for Syft Tensor: 0.025328874588012695 seconds


In [5]:
t0 = time()
tweets_data = sy.Tensor(impressions).private(min_val=500/scale, max_val=25000/scale, entities = data_subjects,ndept=True)
tf = time() - t0

print(f"Time make Private Syft Tensor: {tf} seconds")

what type of child do we have now? [90 90 90 ... 97 97 97] <class 'numpy.ndarray'>
what type of child do we have now? [90 90 90 ... 97 97 97] <class 'numpy.ndarray'>
Time make Private Syft Tensor: 0.0018227100372314453 seconds


In [6]:
domain_node = sy.login(email="info@openmined.org",password="changethis",port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into cicada... done!


In [7]:
%%time
name = f"Tweets data- {time()}"

domain_node.load_dataset(
    assets={name: tweets_data},
    name=name,
    description=" Tweets- 1M rows",
    use_blob_storage=True
)

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/ruchi/PySyft


Loading dataset... checking asset types...                                                                                                                                    

Uploading Tweets data- 1647976670.5323231: 100%|██| 1/1 [00:00<00:00, 37.84it/s]

Loading dataset... uploading...                        

Loading dataset... uploading... SUCCESS!                        

Run <your client variable>.datasets to see your new dataset loaded into your machine!
CPU times: user 63.9 ms, sys: 11.9 ms, total: 75.7 ms
Wall time: 299 ms


In [8]:
domain_node.datasets

Idx,Name,Description,Assets,Id
[0],Tweets data- 1647935659.4255164,Tweets- 1B rows,"[""Tweets data- 1647935659.4255164""] ->",3ccd2351-5a0c-4ea2-8ad4-2a532a2df419
[1],Tweets data- 1647937494.7269194,Tweets- 1B rows,"[""Tweets data- 1647937494.7269194""] ->",679498a5-0555-4d9f-9ea0-342daed6ce6e
[2],Tweets data- 1647937579.130175,Tweets- 1B rows,"[""Tweets data- 1647937579.130175""] ->",b1582714-f88e-405f-a0e5-e577b7b0364c
[3],Tweets data- 1647965464.2495682,Tweets- 1M rows,"[""Tweets data- 1647965464.2495682""] ->",34e569f7-1fc1-4922-b30e-5b471c4808b8
[4],Tweets data- 1647966224.1908379,Tweets- 1B rows,"[""Tweets data- 1647966224.1908379""] ->",0f6da123-b8a5-4c3f-86f1-4d8a9ce5a3f9
[5],Tweets data- 1647967659.5960708,Tweets- 1B rows,"[""Tweets data- 1647967659.5960708""] ->",246a3acd-488c-4ecc-aa28-ca3472e39a73
[6],Tweets data- 1647972025.206142,Tweets- 1B rows,"[""Tweets data- 1647972025.206142""] ->",74f0efc8-9d23-4594-99f8-a3b26d50da63
[7],Tweets data- 1647972492.089469,Tweets- 1B rows,"[""Tweets data- 1647972492.089469""] ->",2449309d-4203-4835-9301-c4013bc89dda
[8],Tweets data- 1647974496.6390357,Tweets- 1B rows,"[""Tweets data- 1647974496.6390357""] ->",a895eeff-6cb7-455b-ab3f-0f8a101c5c93
[9],Tweets data- 1647974905.40028,Tweets- 1M rows,"[""Tweets data- 1647974905.40028""] ->",bdb36738-551e-4421-959f-2faad571aeec


In [9]:
data = domain_node.datasets[-1][name]

In [10]:
data

<TensorPointer -> cicada:f29abcc14a9345e09202a218e21cc7a4>

In [11]:
%%time
sum_result = data.sum()
sum_result.block

CPU times: user 6.57 ms, sys: 7.97 ms, total: 14.5 ms
Wall time: 377 ms


<TensorPointer -> cicada:2fb1d9d0240b4723a6f7b2105a55dac6>

In [12]:
published_result = sum_result.publish(sigma=500_000)

In [13]:
published_result.exists

False

In [14]:
data

<TensorPointer -> cicada:f29abcc14a9345e09202a218e21cc7a4>

In [15]:
c = sum_result.get_copy()

what type of child do we have now? GammaTensor(value=DeviceArray(118914214, dtype=int32), data_subjects=<syft.core.adp.entity_list.EntityList object at 0x7fe4dc293580>, min_val=5000000.0, max_val=250000000.0, is_linear=True, func=<function no_op at 0x7fe4e7d39700>, id='995680785', inputs=DeviceArray([90, 90, 90, ..., 97, 97, 97], dtype=int32), state={'995680785': ...}) <class 'syft.core.tensor.autodp.gamma_tensor.GammaTensor'>
